In [1]:
import tensorflow as tf
import numpy as np
import deepxde as dde

ImportError: initialization failed

In [ ]:
dde.backend.set_default_backend("tensorflow")

Setting the default backend to "tensorflow". You can change it in the ~/.deepxde/config.json file or export the DDE_BACKEND environment variable. Valid options are: tensorflow.compat.v1, tensorflow, pytorch, jax, paddle (all lowercase)


In [ ]:
def psi_real(x,y,t):
    return -1/(2*np.pi)*np.sin(np.pi*(np.power(x,2)+np.power(y,2)))*np.sin(np.power(t,2)+1)

def p_real(x,y,t):
    return np.sin(x-y+t)

In [7]:
DTYPE = 'float32'
pi = tf.constant(np.pi,dtype='float32')

def residual(u,v,u_t,v_t,u_x,u_xx,u_y,u_yy,v_x,v_xx,v_y,v_yy,p_x,p_y,l1, l2, x, y, t):
    
    g_x, g_y = source(x,y,t,l1,l2) 

    f_u = u_t + l1*(u*u_x + v*u_y) + p_x - l2* (u_xx + u_yy) - g_x
    f_v = v_t + l1*(u*v_x + v*v_y) + p_y - l2* (v_xx + v_yy) - g_y

    return f_u, f_v

def source(x,y,t,l1,l2):

    
    r = tf.pow(x,2) + tf.pow(y,2)
    T = tf.pow(t,2) + 1
    g_x = -2*t*y*tf.sin(pi*r)*tf.cos(T) - l1*y*tf.pow(tf.sin(pi*r)*tf.sin(T),2) - 4 * l2* tf.pow(pi,2)*y*r*tf.sin(pi*r)*tf.sin(T) + 8* l2 * pi*y*tf.cos(pi*r)*tf.sin(T) + tf.cos(x-y+t)
    g_y =  2*t*x*tf.sin(pi*r)*tf.cos(T) - l1*x*tf.pow(tf.sin(pi*r)*tf.sin(T),2) + 4 * l2* tf.pow(pi,2)*x*r*tf.sin(pi*r)*tf.sin(T) - 8* l2 * pi*x*tf.cos(pi*r)*tf.sin(T) - tf.cos(x-y+t)

    return g_x, g_y

def fun_u_0(x, y):
    r = tf.pow(x,2) + tf.pow(y,2)
    return tf.constant(-np.sin(1)/(2*np.pi),dtype='float32') * tf.sin(pi*r) 

def fun_u_b(t,x, y):
    n = x.shape[0]
    return tf.zeros((n,1),dtype = DTYPE)


class PhysicsInformedNN:
    def __init__(self, lb, ub, layers, psi0, x0, X):

        self.lb = lb
        self.ub = ub
                
        self.layers = layers

        self.u0 = psi0
        self.X0 = x0
        self.X = X

        self.model = self.initialize_NN(layers)
        self.lambda1 = 1
        self.lambda2 = 1
    def initialize_NN(self,layers):
        model = tf.keras.Sequential()
        model.add(tf.keras.Input(layers[0]))
        scaling_layer = tf.keras.layers.Lambda(
            lambda x: 2.0*(x-self.lb)/(self.ub-self.lb)-1.0)
        model.add(scaling_layer)
        num_layers = len(layers)
        for i in range(1,num_layers-2):
            model.add(tf.keras.layers.Dense(layers[i],
                                            activation=tf.keras.activations.get('tanh'),
                                            kernel_initializer='glorot_normal'))
        model.add(tf.keras.layers.Dense(layers[-1]))

        return model
    

    def loss(self, X, X0, psi0,):
        psi_and_p = self.model(X0)
        psi_pred =  psi_and_p[:,0:1]
        p_pred = psi_and_p[:,1:2]

        loss = tf.reduce_mean(tf.square(psi0-psi_pred))

        r1,r2 = self.get_residual(X)

        phi_ru = tf.reduce_mean(tf.square(r1))
        phi_rv = tf.reduce_mean(tf.square(r2))

        loss += phi_ru 
        loss += phi_rv

        return loss
    
    def get_residual(self,X):
        with tf.GradientTape(persistent=True) as tape:
            x = X[:,0:1]
            y = X[:,1:2]
            t = X[:,2:3]

            tape.watch(x)
            tape.watch(y)
            tape.watch(t)

            psi_and_p = self.model(tf.stack([x[:,0],y[:,0],t[:,0]], axis=1))

            psi = psi_and_p[:,0:1]
            p = psi_and_p[:,1:2]

            u = tape.gradient(psi,y)
            v = -tape.gradient(psi,x) 

            u_t = tape.gradient(u,t)
            v_t = tape.gradient(v,t)

            u_x = tape.gradient(u,x)
            u_xx = tape.gradient(u_x,x)

            u_y = tape.gradient(u,y)
            u_yy = tape.gradient(u_y,y)

            v_x = tape.gradient(v,x)
            v_xx = tape.gradient(v_x,x)

            v_y = tape.gradient(v,y)
            v_yy = tape.gradient(v_y,y)

            p_x = tape.gradient(p,x)
            p_y = tape.gradient(p,y)

        del tape
        
        l1 = self.lambda1
        l2 = self.lambda2
        f_u, f_v = residual(u,v,u_t,v_t,u_x,u_xx,u_y,u_yy,v_x,v_xx,v_y,v_yy,p_x,p_y,l1, l2, x, y, t)

        return f_u, f_v
    
    def predict(self, X):
        with tf.GradientTape(persistent=True) as tape:
            x = X[:,0:1]
            y = X[:,1:2]
            t = X[:,2:3]

            tape.watch(x)
            tape.watch(y)
            tape.watch(t)

            psi_and_p = self.model(tf.stack([x[:,0],y[:,0],t[:,0]], axis=1))

            psi = psi_and_p[:,0:1]
            p = psi_and_p[:,1:2]

            u = tape.gradient(psi,y)
            v = -tape.gradient(psi,x) 
        del tape
        return u, v ,p 
    
    def loss_gradient(self,X,X0,u0, v0):

        with tf.GradientTape(persistent=True) as tape:
            tape.watch(self.model.trainable_variables)
            loss = self.loss(self, X, X0, u0, v0)
            g = tape.gradient(loss, self.model.trainable_variables)

        del tape
        return loss, g

    def train(self):
        def time_step():
            loss = self.loss(self.X, self.X0, self.psi0)
            return loss
        variables = self.model.trainable_variables
        cor=50
        tol=1.0  * np.finfo(float).eps
        iter=50000
        fun=50000
        ls=50
        dde.optimizers.config.set_LBFGS_options(maxcor=cor, ftol=tol,  maxiter=iter, maxfun=fun, maxls=ls)
        dde.optimizers.tfp_optimizer.lbfgs_minimize(variables, time_step)


In [8]:
DTYPE = 'float32'

layers = [3, 20, 20, 20, 20, 20, 20, 20, 20, 2]
layers = [3, 50, 50, 50, 50, 50, 50, 50, 50, 2]
layers = [3, 100, 100, 100, 100, 100, 100, 100, 100, 2]

N_0 = 200
N_b = 100
N_r = 10000

tmin = 0.
tmax = 2.
xmin = -1.
xmax = 1.
ymin = -1
ymax = 1
# Lower bounds
lb = tf.constant([xmin, ymin, tmin], dtype=DTYPE)
# Upper bounds
ub = tf.constant([xmax, ymax, tmax], dtype=DTYPE)

# Set random seed for reproducible results
tf.random.set_seed(0)

# Draw uniform sample points for initial boundary data
t_b = tf.random.uniform((N_b,1), lb[0], ub[0], dtype=DTYPE)
x_b = tf.random.uniform((N_b,1), lb[1], ub[1], dtype=DTYPE)
y_b = np.zeros((N_b,1))


In [10]:
layers = [3, 20, 20, 20, 20, 20, 20, 20, 20, 2]
layers2 = [3, 50, 50, 50, 50, 50, 50, 50, 50, 2]
layers3 = [3, 100, 100, 100, 100, 100, 100, 100, 100, 2]

N_0 = 200
N_b = 100
N_r = 10000

tmin = 0.
tmax = 2.
xmin = -1.
xmax = 1.
ymin = -1
ymax = 1
# Lower bounds
lb = tf.constant([xmin, ymin, tmin], dtype=DTYPE)
# Upper bounds
ub = tf.constant([xmax, ymax, tmax], dtype=DTYPE)

# Set random seed for reproducible results
tf.random.set_seed(0)

# Draw uniform sample points for initial boundary data
t_b = tf.random.uniform((N_b,1), lb[0], ub[0], dtype=DTYPE)
x_b = tf.random.uniform((N_b,1), lb[1], ub[1], dtype=DTYPE)
y_b = np.zeros((N_b,1))
for i in range(x_b.shape[0]):
    y_b[i] = np.power(-1,i)*np.sqrt(1-np.power(x_b[i],2))

y_b = tf.convert_to_tensor(y_b, dtype=DTYPE)

X_b = tf.concat([x_b, y_b , t_b], axis=1)

t_0 = tf.ones((N_0,1), dtype=DTYPE)*lb[0]
x_0 = tf.random.uniform((N_0,1), lb[1], ub[1], dtype=DTYPE)
y_0 = np.zeros((N_0,1))
y_0 = tf.random.uniform((N_0,1),-1,1, dtype=DTYPE)*tf.sqrt(1-tf.pow(x_0,2))

X_0 = tf.concat([x_0, y_0 , t_0], axis=1)

u_0 = fun_u_0(x_0,y_0)

u_b = fun_u_b(t_b,x_b,y_b)

X_data = tf.concat([X_0, X_b],0)
u_data = tf.concat([u_0, u_b],0)

t_r = tf.random.uniform((N_r,1), lb[0], ub[0], dtype=DTYPE)
x_r = tf.random.uniform((N_r,1), lb[1], ub[1], dtype=DTYPE)
y_r = tf.random.uniform((N_r,1),-1,1, dtype=DTYPE)*tf.sqrt(1-tf.pow(x_r,2))
X_r = tf.concat([ x_r, y_r, t_r], axis=1)



In [130]:
model1 = PhysicsInformedNN(lb, ub, layers, u_data, X_data, X_r)

In [131]:
def time_step():
        loss = model1.loss(model1.X, model1.X0, model1.u0)
        return loss

In [132]:
variables1 = model1.model.trainable_variables

In [133]:
cor=100
tol=1.0  * np.finfo(float).eps
iter=100000
fun=100000
ls=100

In [134]:
dde.optimizers.config.set_LBFGS_options(maxcor=cor, ftol=tol,  maxiter=iter, maxfun=fun, maxls=ls)

In [135]:
dde.optimizers.tfp_optimizer.lbfgs_minimize(variables1, time_step)

In [12]:
modelo = tf.saved_model.load("PINN_Export")
modelo2 = tf.saved_model.load("PINN_Export2")

In [13]:
def predict_export(modelo, X):
        with tf.GradientTape(persistent=True) as tape:
            x = X[:,0:1]
            y = X[:,1:2]
            t = X[:,2:3]

            tape.watch(x)
            tape.watch(y)
            tape.watch(t)

            psi_and_p = modelo.serve(tf.stack([x[:,0],y[:,0],t[:,0]], axis=1))

            psi = psi_and_p[:,0:1]
            p = psi_and_p[:,1:2]

            u = tape.gradient(psi,y)
            v = -tape.gradient(psi,x) 
        del tape
        return psi,u, v ,p 

In [14]:
psi_p1  = modelo.serve(X_b)
psi_p2 = modelo2.serve(X_b)

In [21]:
psi_p1[:,0:1]

<tf.Tensor: shape=(100, 1), dtype=float32, numpy=
array([[-0.00022879],
       [-0.01397401],
       [ 0.00938392],
       [ 0.01222411],
       [ 0.00990143],
       [-0.00188887],
       [ 0.01579159],
       [-0.00897533],
       [ 0.00441772],
       [-0.02240348],
       [-0.00039992],
       [ 0.0132367 ],
       [ 0.01288891],
       [ 0.00085759],
       [ 0.00686359],
       [-0.00730622],
       [-0.00605202],
       [ 0.01168314],
       [-0.01353061],
       [-0.02565086],
       [-0.00438219],
       [-0.03291833],
       [-0.00652015],
       [-0.00257194],
       [ 0.00613594],
       [ 0.00050801],
       [-0.00437665],
       [-0.02961528],
       [-0.02477169],
       [-0.00225261],
       [-0.01955342],
       [ 0.0074102 ],
       [-0.00708121],
       [-0.02279162],
       [-0.0152958 ],
       [-0.02119565],
       [ 0.00780365],
       [ 0.00290301],
       [-0.00401568],
       [-0.01898527],
       [-0.01624179],
       [ 0.00079322],
       [-0.0130285 ],
    

In [20]:
psi_p2[:,0:1]

<tf.Tensor: shape=(100, 1), dtype=float32, numpy=
array([[ 0.0039874 ],
       [ 0.00169924],
       [-0.01044133],
       [-0.00136784],
       [-0.00333443],
       [ 0.00643209],
       [ 0.00469127],
       [ 0.00543842],
       [ 0.00644419],
       [ 0.0083209 ],
       [-0.00397006],
       [-0.00021479],
       [ 0.01017579],
       [ 0.00259945],
       [-0.00723222],
       [-0.00215033],
       [-0.00337973],
       [-0.00079259],
       [ 0.00445166],
       [-0.01211187],
       [ 0.00180981],
       [-0.02956793],
       [ 0.00791755],
       [ 0.00025323],
       [ 0.00532827],
       [-0.00027016],
       [-0.00117484],
       [-0.0114471 ],
       [-0.00783458],
       [ 0.00066254],
       [ 0.00630137],
       [ 0.00218585],
       [-0.00557807],
       [ 0.00485465],
       [ 0.00615188],
       [ 0.00043711],
       [-0.00621083],
       [-0.00529853],
       [ 0.00592753],
       [ 0.00128052],
       [ 0.00128147],
       [-0.00235751],
       [ 0.00412413],
    

In [46]:
tf.sin(x_b-y_b+t_b)

<tf.Tensor: shape=(100, 1), dtype=float32, numpy=
array([[-0.9633066 ],
       [-0.30890974],
       [ 0.11675685],
       [ 0.9981049 ],
       [-0.7402039 ],
       [ 0.99462396],
       [-0.94021726],
       [ 0.6694828 ],
       [-0.6172565 ],
       [-0.00179368],
       [ 0.5283813 ],
       [ 0.9999445 ],
       [-0.8065509 ],
       [ 0.91013885],
       [-0.99997455],
       [ 0.9999994 ],
       [-0.5364189 ],
       [ 0.99973005],
       [-0.6767245 ],
       [-0.73193246],
       [ 0.30341506],
       [ 0.43029588],
       [-0.7874243 ],
       [ 0.9750596 ],
       [-0.6550635 ],
       [ 0.9598588 ],
       [ 0.6188211 ],
       [ 0.4560982 ],
       [-0.95276403],
       [ 0.84618783],
       [-0.986841  ],
       [ 0.7927241 ],
       [-0.97871214],
       [ 0.4496697 ],
       [-0.797649  ],
       [ 0.98750454],
       [-0.9389548 ],
       [ 0.25896737],
       [-0.17127714],
       [-0.79993755],
       [-0.2136272 ],
       [ 0.998437  ],
       [-0.5864151 ],
    

In [114]:
u_real = psi_real(x_b,y_b,t_b)

In [118]:
u_r

<tf.Tensor: shape=(100, 1), dtype=float32, numpy=
array([[ 0.02518702],
       [ 0.13850361],
       [ 0.20045686],
       [-0.1077984 ],
       [ 0.12012702],
       [-0.05691618],
       [-0.0560922 ],
       [-0.21326202],
       [ 0.00860298],
       [ 0.04820606],
       [ 0.16235691],
       [-0.07790613],
       [-0.00485533],
       [-0.10916328],
       [-0.14541376],
       [-0.10306966],
       [-0.11556935],
       [-0.09080085],
       [-0.13645285],
       [ 0.09217471],
       [ 0.07929713],
       [-0.11453706],
       [-0.11229664],
       [-0.1220786 ],
       [ 0.00626338],
       [-0.04580086],
       [ 0.14830792],
       [-0.10935807],
       [ 0.02805078],
       [ 0.10190618],
       [-0.09693474],
       [-0.06190869],
       [-0.16600025],
       [-0.07907733],
       [ 0.10724944],
       [-0.10649604],
       [-0.10417783],
       [ 0.11370295],
       [ 0.01026362],
       [ 0.10735679],
       [ 0.0502921 ],
       [-0.1325401 ],
       [ 0.12866795],
    

In [115]:
u_real

array([[ 1.28278614e-08],
       [ 1.35587506e-08],
       [ 1.17455468e-08],
       [ 1.18195462e-08],
       [ 1.19122729e-08],
       [ 1.36587204e-08],
       [ 1.17094103e-08],
       [ 1.26946453e-08],
       [ 1.27272246e-08],
       [ 1.39131489e-08],
       [ 1.28439419e-08],
       [-2.14770477e-08],
       [ 1.18624648e-08],
       [ 1.22262547e-08],
       [ 1.18942385e-08],
       [ 1.21894921e-08],
       [ 1.38231853e-08],
       [-2.14662954e-08],
       [ 1.38035032e-08],
       [ 1.38110794e-08],
       [ 1.29378108e-08],
       [ 8.37897929e-08],
       [ 1.29931204e-08],
       [ 1.33377318e-08],
       [ 8.11978254e-08],
       [ 1.17401324e-08],
       [ 1.32607019e-08],
       [ 1.34863303e-08],
       [ 1.35219933e-08],
       [ 8.53218936e-08],
       [-2.37887274e-08],
       [-2.18507967e-08],
       [ 1.28332545e-08],
       [ 1.38820200e-08],
       [ 1.38074636e-08],
       [ 1.33722109e-08],
       [ 1.19237002e-08],
       [ 1.30290072e-08],
       [ 1.3

In [116]:
u_r

<tf.Tensor: shape=(100, 1), dtype=float32, numpy=
array([[ 0.02518702],
       [ 0.13850361],
       [ 0.20045686],
       [-0.1077984 ],
       [ 0.12012702],
       [-0.05691618],
       [-0.0560922 ],
       [-0.21326202],
       [ 0.00860298],
       [ 0.04820606],
       [ 0.16235691],
       [-0.07790613],
       [-0.00485533],
       [-0.10916328],
       [-0.14541376],
       [-0.10306966],
       [-0.11556935],
       [-0.09080085],
       [-0.13645285],
       [ 0.09217471],
       [ 0.07929713],
       [-0.11453706],
       [-0.11229664],
       [-0.1220786 ],
       [ 0.00626338],
       [-0.04580086],
       [ 0.14830792],
       [-0.10935807],
       [ 0.02805078],
       [ 0.10190618],
       [-0.09693474],
       [-0.06190869],
       [-0.16600025],
       [-0.07907733],
       [ 0.10724944],
       [-0.10649604],
       [-0.10417783],
       [ 0.11370295],
       [ 0.01026362],
       [ 0.10735679],
       [ 0.0502921 ],
       [-0.1325401 ],
       [ 0.12866795],
    